In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers, losses, metrics

# Paramètres communs
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 38

# Préparation des données
def prepare_data(data_dir):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
    
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
    
    # Optimisation des performances
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds

# Modèle VGG19
def create_vgg19_model():
    base_model = applications.VGG19(
        include_top=False,
        weights=None,
        input_shape=(224, 224, 3)
    )
    
    # Ajout des couches de classification
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

# Modèle ResNet34
def create_resnet34_model():
    # Implémentation simplifiée de ResNet34
    def residual_block(x, filters, downsample=False):
        strides = (2, 2) if downsample else (1, 1)
        
        # Branche principale
        y = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(x)
        y = layers.BatchNormalization()(y)
        y = layers.Activation('relu')(y)
        y = layers.Conv2D(filters, kernel_size=3, padding='same')(y)
        y = layers.BatchNormalization()(y)
        
        # Connexion résiduelle
        if downsample:
            x = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(x)
            x = layers.BatchNormalization()(x)
        
        y = layers.add([x, y])
        y = layers.Activation('relu')(y)
        return y
    
    # Architecture ResNet34
    input = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    # Couches résiduelles
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128, downsample=True)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    
    x = residual_block(x, 256, downsample=True)
    x = residual_block(x, 256)
    x = residual_block(x, 256)
    x = residual_block(x, 256)
    x = residual_block(x, 256)
    x = residual_block(x, 256)
    
    x = residual_block(x, 512, downsample=True)
    x = residual_block(x, 512)
    x = residual_block(x, 512)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=input, outputs=predictions)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

# Modèle DenseNet121
def create_densenet121_model():
    base_model = applications.DenseNet121(
        include_top=False,
        weights=None,
        input_shape=(224, 224, 3)
    )
    
    # Ajout des couches de classification
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

In [2]:
# Modèle VGG19 avec poids pré-entraînés
def create_vgg19_pretrained():
    base_model = applications.VGG19(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    
    # Gel des couches de base
    base_model.trainable = False
    
    # Ajout de nouvelles couches de classification
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

# Fonction similaire pour ResNet34 et DenseNet121 avec poids pré-entraînés

In [3]:
def create_efficientnet_model():
    base_model = applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    
    base_model.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    predictions = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=predictions)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
import tensorflow_addons as tfa

def create_vit_model():
    # Paramètres du ViT
    patch_size = 16
    num_patches = (224 // patch_size) ** 2
    projection_dim = 64
    num_heads = 4
    transformer_layers = 8
    
    # Préprocessing: découpage en patches
    inputs = layers.Input(shape=(224, 224, 3))
    patches = layers.Conv2D(
        filters=projection_dim,
        kernel_size=patch_size,
        strides=patch_size,
        padding='valid'
    )(inputs)
    patches = layers.Reshape((num_patches, projection_dim))(patches)
    
    # Ajout du token de classification et positional encoding
    class_token = tf.Variable(tf.random.normal([1, 1, projection_dim]))
    class_tokens = tf.tile(class_token, [tf.shape(patches)[0], 1, 1])
    x = tf.concat([class_tokens, patches], axis=1)
    
    # Positional encoding
    positions = tf.range(start=0, limit=num_patches + 1, delta=1)
    positional_encoding = layers.Embedding(
        input_dim=num_patches + 1,
        output_dim=projection_dim
    )(positions)
    x = x + positional_encoding
    
    # Couches Transformer
    for _ in range(transformer_layers):
        # Self-attention
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=projection_dim // num_heads
        )(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attention_output)
        
        # MLP
        mlp_output = layers.Dense(projection_dim * 2, activation='gelu')(x)
        mlp_output = layers.Dropout(0.1)(mlp_output)
        mlp_output = layers.Dense(projection_dim)(mlp_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + mlp_output)
    
    # Classification head
    x = layers.Lambda(lambda x: x[:, 0])(x)  # Prendre le token de classification
    x = layers.Dense(512, activation='gelu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='gelu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0001),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

ModuleNotFoundError: No module named 'tensorflow_addons'